In [130]:
## Preparação do ambiente
# Carregando as bibliotecas necessárias
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad, odeint
from scipy.interpolate import interp1d
from ipywidgets import Layout, interact, IntSlider

import os as os
DATA_FOLDER = os.path.join(os.getcwd(),'iba_activities')

!git clone https://github.com/tiagofiorini/iba_activities.git

fatal: destination path 'iba_activities' already exists and is not an empty directory.


# Atividade 1A - O poder de discriminação da técnica RBS

## Introdução

O fator cinemático $K$ informa a fração da energia inicial do íon que é retida com ele após uma colisão com um átomo do material alvo, em que o íon incidente sai com um ângulo $\theta$ em relação com sua direção inicial.

$K= \left( \frac{A_1}{A_1+A_2}  \right)^2 \left[ \cos \theta + \sqrt{ \frac{A_2^2}{A_1^2} - \sin^2 \theta } \right]^2 $

É pelo fator cinemátice que fazemos a diferenciação elementar com a técnica RBS. O poder de discriminação é definido como:

$ PD = \frac{E_A - E_{A+1}} {\sigma} = \frac{K(A_1,A_2) - K(A_1,A_2-1)} {\sigma} E_0 = \frac{K(A_1,A_2) - K(A_1,A_2-1)} {F}  $
com $F$ sendo $F = \sigma / E_0$




In [131]:
# Definição da função que calcula o fator cinemático
def K ( A1, A2, theta):
  if A1 < A2:
    K = ( A1**2 / (A1 + A2)**2 ) * (np.cos(theta) + np.sqrt((A2/A1)**2 - np.sin(theta)**2) )**2
  elif np.arcsin(A2/A1) > theta:
    K = ( A1**2 / (A1 + A2)**2 ) * (np.cos(theta) + np.sqrt((A2/A1)**2 - np.sin(theta)**2) )**2
  return K

Vamos tentar descobrir os materiais que compõem as amostras cujos espectros foram simulados, e guardados nos arquivos spec?.dat.

In [ ]:
file_name = 'spec1.dat'

data = np.loadtxt(os.path.join(DATA_FOLDER,file_name))[:,2]

def interactive_func1 ( aA = 12, aB = 5):  
  theta = np.radians(120)
  E0 = 2200
  cmax = 1.1*data.max()
  EA = E0 * K(4,aA,theta)
  EB = E0 * K(4,aB,theta)
  plt.figure(figsize=(10,6))
  plt.vlines(EA,0,cmax,colors='r')
  plt.vlines(EB,0,cmax,colors='g')  
  plt.xlim(xmin=0.0,xmax=2200)
  plt.ylim(ymin=0.0,ymax=cmax)
  plt.step(np.arange(0,len(data),1),data,'k')
  plt.xlabel('Energia (keV)',fontsize=16)
  plt.ylabel('Counts (#)',fontsize=16)
  plt.tick_params(labelsize=14)

interactive_plot = interactive(interactive_func1, aA=IntSlider(5.0,5.0,250.0,1.0,layout=Layout(width='1000px')), aB=IntSlider(5.0,5.0,250.0,1.0,layout=Layout(width='1000px')))
interactive_plot

Agora, vamos tentar descobrir os elementos que compõem a amostra cujo espectro experimental medido está guardado no arquivo R042544P.dat.

In [ ]:
Exp_data = np.loadtxt(os.path.join(DATA_FOLDER,'R042544P.dat'))

def interactive_func2 ( aA = 12, aB = 5):  
  theta = np.radians(120)
  E0 = 2200
  cmax = 1.1*Exp_data[25:].max()
  EA = (E0 * K(4,aA,theta) - 166.733)/5.27912
  EB = (E0 * K(4,aB,theta) - 166.733)/5.27912
  plt.figure(figsize=(10,6))
  plt.vlines(EA,0,cmax,colors='r')
  plt.vlines(EB,0,cmax,colors='g')  
  plt.xlim(xmin=25.0,xmax=1024)
  plt.ylim(ymin=0.0,ymax=cmax)
  plt.step(np.arange(0,1024,1),Exp_data,'k')
  plt.xlabel('Energia (keV)',fontsize=16)
  plt.ylabel('Counts (#)',fontsize=16)
  plt.tick_params(labelsize=14)

interactive_plot = interactive(interactive_func2, aA=(5.0,250.0,1.0), aB=(5.0,250.0,1.0))
interactive_plot

# Atividade 3B - Encontrando proporções com um espectro RBS

## Introdução

A área total de um pico no espectro RBS pode ser aproximada pela expressão:
$ Area = \bar{\sigma_i} (E_0) \cdot \Omega \cdot Q \cdot N_t \cdot \left( \frac{t} {cos \theta}  \right) $

Num mesmo espectro, as áreas de dois picos de elementos diferentes (mas na mesma camada!) pode ser relacionada da seguinde forma:

$ \frac{Area_1}{Area_2} = \frac{\bar{\sigma_1} (E_0)} {\bar{\sigma_2} (E_0)} \cdot \left( \frac{N_1}{N_2} \right)$

Para o caso em que ambos os elementos tenham suas condições de espalhamento no domínio da seção de choque Rutherford (potencial Coulombiano), essa espressão pode ser reescrita como:

$ \frac{Area_1}{Area_2} = \left( \frac{Z_1} {Z_2} \right)^2 \cdot \left( \frac{N_1}{N_2} \right)$

Podemos utilizar essa relação para encontrar as proporções dos elementos diretamente.



In [ ]:
file_name = 'spec2.dat'

data = np.loadtxt(os.path.join(DATA_FOLDER,file_name))[:,2]

def interactive_func1 ( canal_inicial = 12, delta_canal = 0):  

  cmax = 1.1*data.max()
  
  canal_final = len(data)-1 if len(data)-1 < (canal_inicial+delta_canal) else (canal_inicial+delta_canal)
  print(canal_final)

  plt.figure(figsize=(10,6))
  plt.vlines(canal_inicial,0,cmax,colors='r')
  plt.vlines(canal_final,0,cmax,colors='r')  
  plt.fill_between(np.arange(canal_inicial,canal_final,1),0,cmax,color='r',alpha=0.2)

  plt.fill_between( [canal_inicial,canal_final], 0, [data[int(canal_inicial)], data[int(canal_final)]] ,color='r',alpha=0.2)
  
  plt.xlim(xmin=0.0,xmax=2200)
  plt.ylim(ymin=0.0,ymax=cmax)
  plt.step(np.arange(0,len(data),1),data,'k')
  plt.xlabel('Energia (keV)',fontsize=16)
  plt.ylabel('Counts (#)',fontsize=16)
  plt.tick_params(labelsize=14)
  Area = data[int(canal_inicial):int(canal_final)].sum()
  print('Área = %.0f'%(Area))
  Bg = (data[int(canal_inicial)]+data[int(canal_final)])*(canal_final-canal_inicial)/2
  print('Fundo = %.1f'%( Bg ))
  print('Área líquida = %.1f'%(Area-Bg))


interactive_plot = interactive(interactive_func1, canal_inicial=IntSlider(5.0,5.0,canal_final,1.0,layout=Layout(width='1000px')), delta_canal=IntSlider(0.0,0.0,300,1,layout=Layout(width='1000px')))
interactive_plot

Vamos agora, encontrar as proporções entre os elementos em um espectro experimental.

In [ ]:
file_name = 'R042544P.dat'

data = np.loadtxt(os.path.join(DATA_FOLDER,file_name))

def interactive_func1 ( canal_inicial = 12, delta_canal = 0):  

  cmax = 1.1*data[25:].max()
  
  canal_final = len(data)-1 if len(data)-1 < (canal_inicial+delta_canal) else (canal_inicial+delta_canal)
  print(canal_final)

  plt.figure(figsize=(10,6))
  plt.vlines(canal_inicial,0,cmax,colors='r')
  plt.vlines(canal_final,0,cmax,colors='r')  
  plt.fill_between(np.arange(canal_inicial,canal_final,1),0,cmax,color='r',alpha=0.2)

  plt.fill_between( [canal_inicial,canal_final], 0, [data[int(canal_inicial)], data[int(canal_final)]] ,color='r',alpha=0.2)
  
  plt.xlim(xmin=25.0,xmax=1024)
  plt.ylim(ymin=0.0,ymax=cmax)
  plt.step(np.arange(0,len(data),1),data,'k')
  plt.xlabel('Energia (keV)',fontsize=16)
  plt.ylabel('Counts (#)',fontsize=16)
  plt.tick_params(labelsize=14)
  Area = data[int(canal_inicial):int(canal_final)].sum()
  print('Área = %.0f'%(Area))
  Bg = (data[int(canal_inicial)]+data[int(canal_final)])*(canal_final-canal_inicial)/2
  print('Fundo = %.1f'%( Bg ))
  print('Área líquida = %.1f'%(Area-Bg))


interactive_plot = interactive(interactive_func1, canal_inicial=IntSlider(5.0,5.0,canal_final,1.0,layout=Layout(width='1000px')), delta_canal=IntSlider(0.0,0.0,300,1,layout=Layout(width='1000px')))
interactive_plot